# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [8]:
# your code here
import pandas as pd
import numpy as np

tictactoe = pd.read_csv('tic-tac-toe.csv')
tictactoe.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


In [9]:
tictactoe.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TL      958 non-null    object
 1   TM      958 non-null    object
 2   TR      958 non-null    object
 3   ML      958 non-null    object
 4   MM      958 non-null    object
 5   MR      958 non-null    object
 6   BL      958 non-null    object
 7   BM      958 non-null    object
 8   BR      958 non-null    object
 9   class   958 non-null    bool  
dtypes: bool(1), object(9)
memory usage: 68.4+ KB


In [13]:
tictactoe = tictactoe.replace({'x':0,'o':1,'b':-1})
    
tictactoe.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,0,0,0,0,1,1,0,1,1,True
1,0,0,0,0,1,1,1,0,1,True
2,0,0,0,0,1,1,1,1,0,True
3,0,0,0,0,1,1,1,-1,-1,True
4,0,0,0,0,1,1,-1,1,-1,True


In [14]:
from sklearn.preprocessing import MinMaxScaler 

tic_X= tictactoe.drop(columns='class',axis=1)
tic_y=tictactoe['class']
normalizer = MinMaxScaler()
normalizer = normalizer.fit(tic_X)
tic_X_normalized = normalizer.transform(tic_X) 
tic_X_normalized

array([[0.5, 0.5, 0.5, ..., 0.5, 1. , 1. ],
       [0.5, 0.5, 0.5, ..., 1. , 0.5, 1. ],
       [0.5, 0.5, 0.5, ..., 1. , 1. , 0.5],
       ...,
       [1. , 0.5, 1. , ..., 0.5, 1. , 0.5],
       [1. , 0.5, 1. , ..., 0.5, 1. , 0.5],
       [1. , 1. , 0.5, ..., 1. , 0.5, 0.5]])

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [20]:
# your code here
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(tic_X_normalized, tic_y)

model = Sequential([
  Dense(10, activation='relu', input_shape=(9,)), 
  Dense(10, activation='relu'),      
  Dense(2, activation='softmax'),   
])

model.compile(
  optimizer='adam', 
  loss='categorical_crossentropy', 
  metrics=['accuracy'],
)

history = model.fit(
  x_train,
  to_categorical(y_train),
  epochs=5, 
  batch_size=32, 
)

model.evaluate(
  x_test,
  to_categorical(y_test)
)

Epoch 1/5
23/23 [==============================] - 1s 1ms/step - loss: 0.6942 - accuracy: 0.5501
Epoch 2/5
23/23 [==============================] - 0s 2ms/step - loss: 0.6632 - accuracy: 0.6462
Epoch 3/5
23/23 [==============================] - 0s 918us/step - loss: 0.6466 - accuracy: 0.6560
Epoch 4/5
23/23 [==============================] - 0s 2ms/step - loss: 0.6398 - accuracy: 0.6560
Epoch 5/5
8/8 [==============================] - 0s 2ms/step - loss: 0.6290 - accuracy: 0.6458


[0.6289590001106262, 0.6458333134651184]

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [21]:
# your code here
predictions = model.predict(x_test[:10])
print(np.argmax(predictions, axis=1)) 

1/1 [==============================] - 0s 98ms/step
[1 1 1 1 1 1 1 1 1 1]


In [22]:
y_test[:10]

748    False
373     True
40      True
956    False
169     True
598     True
158     True
521     True
364     True
388     True
Name: class, dtype: bool

## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [25]:
# your code here
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

tictactoe_model = Sequential([
  Dense(10, activation='relu'),
  Dense(10, activation='relu'),
  Dense(2, activation='softmax'),   
])

opt = tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name='Adam'
)


tictactoe_model.compile(
  optimizer=opt,
  loss='sparse_categorical_crossentropy',
  metrics=['accuracy'],
)

history = tictactoe_model.fit(
  x_train,
  y_train,
  epochs=50,
  batch_size=1,
)

tictactoe_model.evaluate(
  x_test,
  y_test
)

Epoch 1/50
718/718 [==============================] - 1s 1ms/step - loss: 0.6538 - accuracy: 0.6574
Epoch 2/50
718/718 [==============================] - 1s 1ms/step - loss: 0.6318 - accuracy: 0.6560
Epoch 3/50
718/718 [==============================] - 1s 1ms/step - loss: 0.6121 - accuracy: 0.6574
Epoch 4/50
718/718 [==============================] - 1s 913us/step - loss: 0.5947 - accuracy: 0.6616
Epoch 5/50
718/718 [==============================] - 1s 966us/step - loss: 0.5716 - accuracy: 0.6643
Epoch 6/50
718/718 [==============================] - 1s 938us/step - loss: 0.5547 - accuracy: 0.6866
Epoch 7/50
718/718 [==============================] - 1s 986us/step - loss: 0.5401 - accuracy: 0.6936
Epoch 8/50
718/718 [==============================] - 1s 970us/step - loss: 0.5242 - accuracy: 0.7047
Epoch 9/50
718/718 [==============================] - 1s 966us/step - loss: 0.5175 - accuracy: 0.7187
Epoch 10/50
718/718 [==============================] - 1s 960us/step - loss: 0.5095 - ac

[0.4337320029735565, 0.7791666388511658]

**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# your answer here
#Adding epoch, reduce batch size and elevate relu.